In [10]:
import chess
import chess.pgn
import pandas as pd
import numpy as np
import xgboost as xgb

Data is from July 2014 with over 1 million games.

In [2]:
from read_pgn import process_pgn_file

In [3]:
# load the first 100,000 games for now
games_data = process_pgn_file('lichess_db_standard_rated_2014-07.pgn', target_move=20, max_games=100000)

In [4]:
df = pd.DataFrame(games_data)
df.head()

,white_material,black_material,material_diff,white_pawns,black_pawns,white_knights,black_knights,white_bishops,black_bishops,white_rooks,...,black_king_attackers,to_move,legal_moves,is_check,is_checkmate,is_stalemate,white_elo,black_elo,elo_diff,result
0,31,29,2,6,7,1,0,1,1,2,...,0,1,40,0,0,0,1525,1458,67,0.0
1,28,28,0,6,6,0,0,1,1,2,...,0,1,32,0,0,0,1467,1314,153,1.0
2,21,29,-8,5,4,1,1,1,1,2,...,0,1,27,0,0,0,1445,1497,-52,0.0
3,29,29,0,4,7,0,0,2,1,2,...,0,1,5,1,0,0,1752,1698,54,1.0
4,29,22,7,7,5,0,0,1,1,2,...,0,1,32,0,0,0,1451,1415,36,1.0


In [6]:
df.columns

Index(['white_material', 'black_material', 'material_diff', 'white_pawns',
       'black_pawns', 'white_knights', 'black_knights', 'white_bishops',
       'black_bishops', 'white_rooks', 'black_rooks', 'white_queens',
       'black_queens', 'white_king_attackers', 'black_king_attackers',
       'to_move', 'legal_moves', 'is_check', 'is_checkmate', 'is_stalemate',
       'white_elo', 'black_elo', 'elo_diff', 'result'],
      dtype='object')

In [9]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['result'])
y = df['result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1031)

In [12]:
X_train

,white_material,black_material,material_diff,white_pawns,black_pawns,white_knights,black_knights,white_bishops,black_bishops,white_rooks,...,white_king_attackers,black_king_attackers,to_move,legal_moves,is_check,is_checkmate,is_stalemate,white_elo,black_elo,elo_diff
50285,38,38,0,7,7,2,2,2,2,2,...,0,0,1,32,0,0,0,1643,1836,-193
50189,34,34,0,6,6,2,2,1,1,2,...,0,0,1,44,0,0,0,1926,1955,-29
48395,26,24,2,7,5,1,2,2,1,2,...,0,0,1,44,0,0,0,1501,1299,202
56510,32,32,0,7,7,2,2,0,0,2,...,0,0,1,31,0,0,0,1773,1582,191
92242,32,32,0,7,7,0,1,2,1,2,...,0,0,1,37,0,0,0,1720,1839,-119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86999,19,18,1,6,5,1,1,0,0,2,...,0,0,1,30,0,0,0,1844,1811,33
54944,34,28,6,6,5,2,2,1,1,2,...,0,0,1,44,0,0,0,1776,1890,-114
49279,30,30,0,8,8,0,1,1,0,2,...,0,0,1,25,0,0,0,1660,1589,71
10136,31,31,0,6,6,0,1,2,1,2,...,0,0,1,44,0,0,0,1762,1785,-23


In [24]:
model = xgb.XGBRegressor(objective='reg:logistic', n_estimators=100)
model.fit(X_train, y_train)

,objective,'reg:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [25]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R squared: {r2}")

RMSE: 0.19079551861843164
R squared: 0.20195930861646783


In [26]:
y_pred

array([0.37880844, 0.8222882 , 0.55940527, ..., 0.31399167, 0.3902438 ,
       0.88142294], shape=(20000,), dtype=float32)

In [27]:
y_pred.min()

np.float32(7.082543e-05)

In [28]:
y_pred.max()

np.float32(0.9989743)

In [33]:
np.unique(y_test)

array([0. , 0.5, 1. ])

In [34]:
thresh_low = 0.4
thresh_high = 0.6

conditions = [y_pred < thresh_low, y_pred > thresh_high]
choices = [0, 1]

y_pred_classes = np.select(conditions, choices, default=0.5)

print(y_pred_classes)

[0.  1.  0.5 ... 0.  0.  1. ]


In [37]:
# Convert 0, 0.5, 1   ->   0, 1, 2
y_test_int = (y_test * 2).astype(int)
y_pred_classes_int = (y_pred_classes * 2).astype(int)

In [38]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test_int, y_pred_classes_int)
print(cm)

[[4805 2844 1723]
 [ 274  310  282]
 [1688 2735 5339]]


In [39]:
from sklearn.metrics import classification_report

report = classification_report(y_test_int, y_pred_classes_int, target_names=['Loss (0)', 'Draw (1)', 'Win (2)'])
print(report)

              precision    recall  f1-score   support

    Loss (0)       0.71      0.51      0.60      9372
    Draw (1)       0.05      0.36      0.09       866
     Win (2)       0.73      0.55      0.62      9762

    accuracy                           0.52     20000
   macro avg       0.50      0.47      0.44     20000
weighted avg       0.69      0.52      0.59     20000

